In [ ]:
!pip install qiskit

In [ ]:
import os
from os.path import join, isfile
import csv

from qiskit import QuantumCircuit, transpile
from qiskit.qasm2 import dumps

In [ ]:
def qiskit_optimize_ibm(fn, output_dir: str):
    circ = QuantumCircuit.from_qasm_file(fn)
    
    optimized_circ = transpile(
        circ, basis_gates=["rz", "cx", "h", "x"], optimization_level=3
    )
   
    count_dict = optimized_circ.count_ops()
    output_path = join(output_dir, os.path.basename(fn))

    with open(output_path, "w") as f:
        f.write(dumps(optimized_circ))

    return (
        sum(count_dict.values()),
        count_dict.get("cx", 0),
        optimized_circ.depth(),
    )


circuit_order = [
    "tof_3",
    "barenco_tof_3",
    "mod5_4",
    "tof_4",
    "barenco_tof_4",
    "tof_5",
    "mod_mult_55",
    "vbe_adder_3",
    "barenco_tof_5",
    "csla_mux_3",
    "rc_adder_6",
    "gf2^4_mult",
    "tof_10",
    "mod_red_21",
    "hwb6",
    "gf2^5_mult",
    "csum_mux_9",
    "barenco_tof_10",
    "qcla_com_7",
    "ham15-low",
    "gf2^6_mult",
    "qcla_adder_10",
    "gf2^7_mult",
    "gf2^8_mult",
    "qcla_mod_7",
    "adder_8",
]

path = "../circuits/nam/"

fns = [fn for fn in os.listdir(path) if isfile(join(path, fn)) and fn[-4:] == "qasm"]

output_dir = "./output/qiskit_nam"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

result_dict = {}

for i, fn in enumerate(fns):
    if fn[:-5] not in circuit_order:
        continue
    gate_cnt, cnot_cnt, depth = qiskit_optimize_ibm(path + fn, output_dir)
    result_dict[fn] = [gate_cnt, cnot_cnt, depth]

csv_file = "./metricks/qiskit_nam.csv"

with open(csv_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "total_gates", "cx_gates", "depth"])
    for fn in circuit_order:
        if fn + ".qasm" in result_dict:
            writer.writerow([fn + ".qasm"] + result_dict[fn + ".qasm"])